# Download & Engineer Los Angeles Collisions Data
### Tiffany Chan
<br>
The purpose of this notebook is to download 2024 collision data from the Los Angeles Open Data portal. This dataset is engineered to add fields and edit existing fields, before being exporting the data to a CSV file.

---
### Part I: Set Up & Read Dataset

In [1]:
import pandas as pd
from datetime import date, datetime

Return the current local date

In [2]:
today = date.today()
print("Run date: ", today)

Run date:  2025-01-28


Get download URL

In [3]:
url = 'https://data.lacity.org/api/views/d5tf-ez2w/rows.csv?accessType=DOWNLOAD'

Read the data

In [4]:
df = pd.read_csv(url)
df.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Location
0,190319651,08/24/2019,08/24/2019,450,3,Southwest,356,997,TRAFFIC COLLISION,3036 3004 3026 3101 4003,22.0,M,H,101.0,STREET,JEFFERSON BL,NORMANDIE AV,"(34.0255, -118.3002)"
1,190319680,08/30/2019,08/30/2019,2320,3,Southwest,355,997,TRAFFIC COLLISION,3037 3006 3028 3030 3039 3101 4003,30.0,F,H,101.0,STREET,JEFFERSON BL,W WESTERN,"(34.0256, -118.3089)"
2,190413769,08/25/2019,08/25/2019,545,4,Hollenbeck,422,997,TRAFFIC COLLISION,3101 3401 3701 3006 3030,NaN,M,X,101.0,STREET,N BROADWAY,W EASTLAKE AV,"(34.0738, -118.2078)"
3,190127578,11/20/2019,11/20/2019,350,1,Central,128,997,TRAFFIC COLLISION,0605 3101 3401 3701 3011 3034,21.0,M,H,101.0,STREET,1ST,CENTRAL,"(34.0492, -118.2391)"
4,190319695,08/30/2019,08/30/2019,2100,3,Southwest,374,997,TRAFFIC COLLISION,0605 4025 3037 3004 3025 3101,49.0,M,B,101.0,STREET,MARTIN LUTHER KING JR,ARLINGTON AV,"(34.0108, -118.3182)"


---
### Part II: Data Engineering

#### 1) Create timestamp
Use "Date Occured field"

In [5]:
timestamp = df['Date Occurred']
timestamp = pd.to_datetime(timestamp)

#### 2) Filter to 2024
Create a new field for the year

In [6]:
year = []

for t in timestamp:
    yr = t.year
    year.append(yr)

df.insert(0,"Year", year)

Filter for only collisions in 2024

In [7]:
df = df[df['Year'] == 2024]
df.head()

,Year,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Location
19678,2024,240406645,03/13/2024,03/10/2024,1900,4,Hollenbeck,412,997,TRAFFIC COLLISION,0605 3030 4024,31.0,M,C,101.0,STREET,200 W AVENUE 26,NaN,"(34.0786, -118.2174)"
45031,2024,241307843,03/28/2024,03/27/2024,2340,13,Newton,1321,997,TRAFFIC COLLISION,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25.0,M,O,102.0,SIDEWALK,ADAMS,GRAND,"(34.0265, -118.2723)"
45311,2024,241214094,08/05/2024,08/05/2024,835,12,77th Street,1256,997,TRAFFIC COLLISION,NaN,30.0,M,H,101.0,STREET,FLORENCE,GRAND,"(33.9747, -118.2803)"
47337,2024,240110106,04/09/2024,04/09/2024,950,1,Central,119,997,TRAFFIC COLLISION,3401 3701 0605 3004 3026 3029 3033 3037 3101 4024,32.0,M,O,101.0,STREET,VIGNES,BAUCHET,"(34.0589, -118.2323)"
68491,2024,240115072,07/21/2024,07/21/2024,235,1,Central,195,997,TRAFFIC COLLISION,0605 3004 3025 3029 3035 3036 3101 4024 3401 3701,25.0,M,H,101.0,STREET,17TH ST,LOS ANGELES ST,"(34.0323, -118.2621)"


Remove the year column to remove clutter

In [8]:
df.drop('Year', axis=1, inplace=True)

#### 3) Add latitude & longitude
Separate "Location" field into latitude & longitude

In [9]:
lat = []
for n in df['Location']:
    split = n.split(', ')
    first_num = split[0]
    latitude = first_num[1:]
    lat.append(latitude)
    
lon = []
for n in df['Location']:
    split = n.split(', ')
    second_num = split[1]
    longitude = second_num[:-1]
    lon.append(longitude)

Create two new fields for storing the data & delete the "Location" field

In [10]:
df.insert(18, "Latitude", lat)
df.insert(19, "Longitude", lon)

df.drop('Location', axis=1, inplace=True)
df.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Latitude,Longitude
19678,240406645,03/13/2024,03/10/2024,1900,4,Hollenbeck,412,997,TRAFFIC COLLISION,0605 3030 4024,31.0,M,C,101.0,STREET,200 W AVENUE 26,NaN,34.0786,-118.2174
45031,241307843,03/28/2024,03/27/2024,2340,13,Newton,1321,997,TRAFFIC COLLISION,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25.0,M,O,102.0,SIDEWALK,ADAMS,GRAND,34.0265,-118.2723
45311,241214094,08/05/2024,08/05/2024,835,12,77th Street,1256,997,TRAFFIC COLLISION,NaN,30.0,M,H,101.0,STREET,FLORENCE,GRAND,33.9747,-118.2803
47337,240110106,04/09/2024,04/09/2024,950,1,Central,119,997,TRAFFIC COLLISION,3401 3701 0605 3004 3026 3029 3033 3037 3101 4024,32.0,M,O,101.0,STREET,VIGNES,BAUCHET,34.0589,-118.2323
68491,240115072,07/21/2024,07/21/2024,235,1,Central,195,997,TRAFFIC COLLISION,0605 3004 3025 3029 3035 3036 3101 4024 3401 3701,25.0,M,H,101.0,STREET,17TH ST,LOS ANGELES ST,34.0323,-118.2621


#### 4) Add month of the year
Derive from "Date Occurred" field

In [11]:
timestamp = df['Date Occurred']
timestamp = pd.to_datetime(timestamp)

month = []

# get month from timestamp
for t in timestamp:
    m = t.month_name()
    month.append(m)

# add month data to dataframe
df.insert(3, "Month", month)
df.head()

,DR Number,Date Reported,Date Occurred,Month,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Latitude,Longitude
19678,240406645,03/13/2024,03/10/2024,March,1900,4,Hollenbeck,412,997,TRAFFIC COLLISION,0605 3030 4024,31.0,M,C,101.0,STREET,200 W AVENUE 26,NaN,34.0786,-118.2174
45031,241307843,03/28/2024,03/27/2024,March,2340,13,Newton,1321,997,TRAFFIC COLLISION,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25.0,M,O,102.0,SIDEWALK,ADAMS,GRAND,34.0265,-118.2723
45311,241214094,08/05/2024,08/05/2024,August,835,12,77th Street,1256,997,TRAFFIC COLLISION,NaN,30.0,M,H,101.0,STREET,FLORENCE,GRAND,33.9747,-118.2803
47337,240110106,04/09/2024,04/09/2024,April,950,1,Central,119,997,TRAFFIC COLLISION,3401 3701 0605 3004 3026 3029 3033 3037 3101 4024,32.0,M,O,101.0,STREET,VIGNES,BAUCHET,34.0589,-118.2323
68491,240115072,07/21/2024,07/21/2024,July,235,1,Central,195,997,TRAFFIC COLLISION,0605 3004 3025 3029 3035 3036 3101 4024 3401 3701,25.0,M,H,101.0,STREET,17TH ST,LOS ANGELES ST,34.0323,-118.2621


#### 5) Add hour of the day
Derive from "Time Occurred" field

In [12]:
hour = (df['Time Occurred']/100).astype(int)
df.insert(4, "Hour", hour)
df.head()

,DR Number,Date Reported,Date Occurred,Month,Hour,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,...,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Latitude,Longitude
19678,240406645,03/13/2024,03/10/2024,March,19,1900,4,Hollenbeck,412,997,...,0605 3030 4024,31.0,M,C,101.0,STREET,200 W AVENUE 26,NaN,34.0786,-118.2174
45031,241307843,03/28/2024,03/27/2024,March,23,2340,13,Newton,1321,997,...,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25.0,M,O,102.0,SIDEWALK,ADAMS,GRAND,34.0265,-118.2723
45311,241214094,08/05/2024,08/05/2024,August,8,835,12,77th Street,1256,997,...,NaN,30.0,M,H,101.0,STREET,FLORENCE,GRAND,33.9747,-118.2803
47337,240110106,04/09/2024,04/09/2024,April,9,950,1,Central,119,997,...,3401 3701 0605 3004 3026 3029 3033 3037 3101 4024,32.0,M,O,101.0,STREET,VIGNES,BAUCHET,34.0589,-118.2323
68491,240115072,07/21/2024,07/21/2024,July,2,235,1,Central,195,997,...,0605 3004 3025 3029 3035 3036 3101 4024 3401 3701,25.0,M,H,101.0,STREET,17TH ST,LOS ANGELES ST,34.0323,-118.2621


#### 6) Add day of week
Derive from "Date Occurred" field

In [13]:
timestamp = df['Date Occurred']
timestamp = pd.to_datetime(timestamp)

day_of_week = []

# get day of week from timestamp
for t in timestamp:
    day = t.day_name()
    day_of_week.append(day)

# add day of week data to dataframe
df.insert(3, "Day Of Week", day_of_week)
df.head()

,DR Number,Date Reported,Date Occurred,Day Of Week,Month,Hour,Time Occurred,Area ID,Area Name,Reporting District,...,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Latitude,Longitude
19678,240406645,03/13/2024,03/10/2024,Sunday,March,19,1900,4,Hollenbeck,412,...,0605 3030 4024,31.0,M,C,101.0,STREET,200 W AVENUE 26,NaN,34.0786,-118.2174
45031,241307843,03/28/2024,03/27/2024,Wednesday,March,23,2340,13,Newton,1321,...,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25.0,M,O,102.0,SIDEWALK,ADAMS,GRAND,34.0265,-118.2723
45311,241214094,08/05/2024,08/05/2024,Monday,August,8,835,12,77th Street,1256,...,NaN,30.0,M,H,101.0,STREET,FLORENCE,GRAND,33.9747,-118.2803
47337,240110106,04/09/2024,04/09/2024,Tuesday,April,9,950,1,Central,119,...,3401 3701 0605 3004 3026 3029 3033 3037 3101 4024,32.0,M,O,101.0,STREET,VIGNES,BAUCHET,34.0589,-118.2323
68491,240115072,07/21/2024,07/21/2024,Sunday,July,2,235,1,Central,195,...,0605 3004 3025 3029 3035 3036 3101 4024 3401 3701,25.0,M,H,101.0,STREET,17TH ST,LOS ANGELES ST,34.0323,-118.2621


---
### Part III: Export Data

In [14]:
collision_data = 'traffic_collisions_' + str(today) + '.csv'
df.to_csv(collision_data, index=False)
print('CSV file has been exported')

CSV file has been exported
